<a href="https://colab.research.google.com/github/estefaniahernandezz/PROYECTO-IA/blob/main/03_modelo_con_OneHot_completo_y_LightGBM_(pip_install).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# NOTEBOOK ALTERNATIVO – INTENTO CON LIGHTGBM / CATBOOST
# ============================================================

# Intento de instalación (falló en Colab por RAM / compatibilidad)
!pip install lightgbm
!pip install catboost

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

print("Cargando archivos...")

train = pd.read_csv("/content/drive/MyDrive/ARCHIVOS IA/train_preprocesado.csv")
test  = pd.read_csv("/content/test.csv")

print("Train:", train.shape, "Test:", test.shape)

# ============================================================
# 1) Reconstruir target
# ============================================================

target_cols = [
    "RENDIMIENTO_GLOBAL_alto",
    "RENDIMIENTO_GLOBAL_bajo",
    "RENDIMIENTO_GLOBAL_medio-alto",
    "RENDIMIENTO_GLOBAL_medio-bajo"
]

train["RENDIMIENTO_GLOBAL"] = train[target_cols].idxmax(axis=1)
train["RENDIMIENTO_GLOBAL"] = train["RENDIMIENTO_GLOBAL"].str.replace("RENDIMIENTO_GLOBAL_", "")

y = train["RENDIMIENTO_GLOBAL"]
X = train.drop(target_cols + ["RENDIMIENTO_GLOBAL"], axis=1)

print("Columnas en X:", X.shape[1])

# ============================================================
# 2) A diferencia del notebook principal:
#    AQUÍ NO LIMITAMOS FILAS (error)
# ============================================================

full = pd.concat([X, test], axis=0)
print("Full concatenado:", full.shape)

# Rellenar nulos
for c in full.select_dtypes(include=['int64','float64']).columns:
    full[c] = full[c].fillna(full[c].median())

for c in full.select_dtypes(include=['object']).columns:
    full[c] = full[c].fillna("Desconocido")

# ============================================================
# 3) One-Hot Encoding gigante
#    (Aquí fue donde explotó Colab)
# ============================================================

print("Aplicando get_dummies... cuidado con la RAM")

try:
    full_enc = pd.get_dummies(full, drop_first=False)
    print("Shape codificado:", full_enc.shape)
except Exception as e:
    print("ERROR durante get_dummies:", e)

# Separar entrenamiento / test
try:
    X_enc = full_enc.iloc[:len(X)]
    test_enc = full_enc.iloc[len(X):]
except:
    print("No fue posible separar matrices (memoria insuficiente).")

# Codificar etiquetas
le = LabelEncoder()
y_enc = le.fit_transform(y)

# ============================================================
# 4) Intento de modelo LIGHTGBM (falló por RAM)
# ============================================================

try:
    import lightgbm as lgb

    model_lgb = lgb.LGBMClassifier(
        n_estimators=300,
        max_depth=-1,
        learning_rate=0.05
    )

    print("Entrenando LightGBM...")
    model_lgb.fit(X_enc, y_enc)

except Exception as e:
    print("ERROR entrenando LightGBM:", e)

# ============================================================
# 5) Intento de modelo CATBOOST (falló por tiempo de ejecución)
# ============================================================

try:
    from catboost import CatBoostClassifier

    model_cb = CatBoostClassifier(
        iterations=300,
        learning_rate=0.1,
        depth=6,
        verbose=50
    )

    print("Entrenando CatBoost...")
    model_cb.fit(X_enc, y_enc)

except Exception as e:
    print("ERROR entrenando CatBoost:", e)

# ============================================================
# Fin del intento alternativo
# ============================================================

print("Notebook alternativo finalizó con errores (esperado).")
